# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [76]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [2]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
#call function defined above to load all filepath names into a list
song_files = get_files('data/song_data')

In [56]:
#full the first entry in the list as filepath
filepath = song_files[0]

In [57]:
#generate pandas dataframe containing songfile infromation
df = pd.read_json(filepath, lines = True)
df.head()

num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [58]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [59]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [60]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [61]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [62]:
log_files = get_files('data/log_data')

In [63]:
filepath = log_files[0]

In [64]:
df = pd.read_json(filepath, lines = True)
df.head()

artist       auth firstName gender  itemInSession lastName     length  \
0     None  Logged In    Walter      M              0     Frye        NaN   
1     None  Logged In    Kaylee      F              0  Summers        NaN   
2  Des'ree  Logged In    Kaylee      F              1  Summers  246.30812   
3     None  Logged In    Kaylee      F              2  Summers        NaN   
4  Mr Oizo  Logged In    Kaylee      F              3  Summers  144.03873   

  level                           location method      page   registration  \
0  free  San Francisco-Oakland-Hayward, CA    GET      Home  1540919166796   
1  free        Phoenix-Mesa-Scottsdale, AZ    GET      Home  1540344794796   
2  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   
3  free        Phoenix-Mesa-Scottsdale, AZ    GET   Upgrade  1540344794796   
4  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   

   sessionId          song  status             ts  \
0         38          None     200  1541105830796   
1        139          None     200  1541106106796   
2        139  You Gotta Be     200  1541106106796   
3        139          None     200  1541106132796   
4        139       Flat 55     200  1541106352796   

                                           userAgent  userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      39  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
3  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [65]:
#create new dataframe filtered to page = 'NextSong'
df = df.query('page == "NextSong"')
df.head()

artist       auth firstName gender  itemInSession lastName  \
2            Des'ree  Logged In    Kaylee      F              1  Summers   
4            Mr Oizo  Logged In    Kaylee      F              3  Summers   
5         Tamba Trio  Logged In    Kaylee      F              4  Summers   
6     The Mars Volta  Logged In    Kaylee      F              5  Summers   
7  Infected Mushroom  Logged In    Kaylee      F              6  Summers   

      length level                     location method      page  \
2  246.30812  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
4  144.03873  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
5  177.18812  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
6  380.42077  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
7  440.26730  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   

    registration  sessionId                          song  status  \
2  1540344794796        139                  You Gotta Be     200   
4  1540344794796        139                       Flat 55     200   
5  1540344794796        139  Quem Quiser Encontrar O Amor     200   
6  1540344794796        139                     Eriatarka     200   
7  1540344794796        139               Becoming Insane     200   

              ts                                          userAgent  userId  
2  1541106106796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
4  1541106352796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
5  1541106496796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
6  1541106673796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
7  1541107053796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

In [66]:
#use pandas dataframe function to_datetime to convert from ms to date time

t = pd.to_datetime(df['ts'], unit = 'ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [67]:
#use the pandas series datetime like properties function to access values
time_data = ([i, i.hour, i.day, i.week, i.month, i.year, i.dayofweek] for i in t)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [68]:
time_df = pd.DataFrame(time_data, columns = column_labels)
time_df.head()


start_time  hour  day  week  month  year  weekday
0 2018-11-01 21:01:46.796    21    1    44     11  2018        3
1 2018-11-01 21:05:52.796    21    1    44     11  2018        3
2 2018-11-01 21:08:16.796    21    1    44     11  2018        3
3 2018-11-01 21:11:13.796    21    1    44     11  2018        3
4 2018-11-01 21:17:33.796    21    1    44     11  2018        3

In [69]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_time  11 non-null     datetime64[ns]
 1   hour        11 non-null     int64         
 2   day         11 non-null     int64         
 3   week        11 non-null     int64         
 4   month       11 non-null     int64         
 5   year        11 non-null     int64         
 6   weekday     11 non-null     int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 744.0 bytes


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [70]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [71]:
user_df = df[['userId','firstName','lastName','gender','level']].drop_duplicates()
user_df.head()

userId firstName lastName gender level
2        8    Kaylee  Summers      F  free
10      10    Sylvie     Cruz      F  free
12      26      Ryan    Smith      M  free
14     101    Jayden      Fox      M  free

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [72]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [77]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit = 'ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [79]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.